<a href="https://colab.research.google.com/github/sfhorng/AIPI-531-Final-Project/blob/main/H_and_M_Data/H%26M_Recommenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AIPI 531 Final Project - Diarra Bell and Stephanie Horng

This notebook trains the SNQN/GRU4Rec models using the H&M Kaggle Dataset: https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/data

Link to Github Repo: https://github.com/sfhorng/AIPI-531-Final-Project/tree/main/Retail_Rocket_Data

### 1. Install Dependencies and Imports

In [1]:
#import libraries
import os
import getpass
import tensorflow as tf

In [2]:
! pip install -Uq kaggle

In [3]:
! git clone https://github.com/sfhorng/AIPI-531-Final-Project.git

Cloning into 'AIPI-531-Final-Project'...
remote: Enumerating objects: 342, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 342 (delta 130), reused 141 (delta 77), pack-reused 128
Receiving objects: 100% (342/342), 77.24 MiB | 47.33 MiB/s, done.
Resolving deltas: 100% (191/191), done.


In [4]:
# Install dependencies from requirements.txt file
repo_name = "AIPI-531-Final-Project" 
REPO_PATH = os.path.join(os.getcwd(), repo_name)
%pip install -r {os.path.join(REPO_PATH, 'requirements.txt')}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 3.1 MB/s eta 0:00:00


### 2. Download and Initialize H&M dataset

In [5]:
HM_DIR = f'{REPO_PATH}/H_and_M_Data'

In [6]:
# Upload kaggle.json credentials for downloading with the API
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"diarrabell","key":"ae3140768a2f7c513228f073eb9ec167"}'}

In [7]:
# make kaggle directory
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Because there are extremely large files in this dataset, we are only downloading files necessary for the project.

In [8]:
! kaggle competitions download h-and-m-personalized-fashion-recommendations -f articles.csv -p '{HM_DIR}/data'

  0% 0.00/4.26M [00:00<?, ?B/s]
100% 4.26M/4.26M [00:00<00:00, 85.5MB/s]


In [9]:
! unzip /content/AIPI-531-Final-Project/H_and_M_Data/data/articles.csv.zip -d /content/AIPI-531-Final-Project/H_and_M_Data/data

Archive:  /content/AIPI-531-Final-Project/H_and_M_Data/data/articles.csv.zip
  inflating: /content/AIPI-531-Final-Project/H_and_M_Data/data/articles.csv  


In [10]:
! kaggle competitions download h-and-m-personalized-fashion-recommendations -f transactions_train.csv -p '{HM_DIR}/data'

 98% 573M/584M [00:03<00:00, 189MB/s]
100% 584M/584M [00:03<00:00, 179MB/s]


In [11]:
! unzip /content/AIPI-531-Final-Project/H_and_M_Data/data/transactions_train.csv.zip -d /content/AIPI-531-Final-Project/H_and_M_Data/data

Archive:  /content/AIPI-531-Final-Project/H_and_M_Data/data/transactions_train.csv.zip
  inflating: /content/AIPI-531-Final-Project/H_and_M_Data/data/transactions_train.csv  


In [12]:
# Create sessions, create popularity dictionary and replay buffer
# Because the H&M dataset is so large, we are sampling the total number of sessions (events)
!python "{HM_DIR}/src/main.py" --data_directory='{HM_DIR}/data' --num_total_sessions=10000

2023-05-05 00:44:03.839252: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Started to read and modify transactions_train
events df items: 104547
There are 104547 total items from transactions_train
item properties items: 105542
There are 1103326 total sessions after removing sessions with less than 3 interactions
Started filtering for the input number of sessions, 10000
Finished creating the sorted_events file
Started to split the sorted events into train, validation, and test
Finished splitting into train, validation, and test sessions
/content/AIPI-531-Final-Project/H_and_M_Data/src/item.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_properties_df['article_id'] = item_encoder.transform

In [13]:
%cd '{HM_DIR}/src'

/content/AIPI-531-Final-Project/H_and_M_Data/src


### 3. Train SNQN model (without item features)

In [14]:
with tf.device('/GPU:0'):
  ! python SNQN_new.py --model=GRU --epoch=10 --data='{HM_DIR}/data'

2023-05-05 00:45:30.919896: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/AIPI-531-Final-Project/H_and_M_Data/src/SNQN_new.py:82: UserWarning: `tf.nn.rnn_cell.GRUCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.GRUCell`, and will be replaced by that in Tensorflow 2.0.
  tf.compat.v1.nn.rnn_cell.GRUCell(self.hidden_size),
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/content/AIPI-531-Final-Project/H_and_M_Data/src/SNQN_new.py:209: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  self.output1 = tf.compat.v1.lay

### 4. Train GRU4Rec model (with item features)

In [15]:
with tf.device('/GPU:0'):
  ! python SNQN_with_features.py --model=GRU --epoch=10 --data='{HM_DIR}/data' 

2023-05-05 01:21:53.943696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/AIPI-531-Final-Project/H_and_M_Data/src/SNQN_with_features.py:83: UserWarning: `tf.nn.rnn_cell.GRUCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.GRUCell`, and will be replaced by that in Tensorflow 2.0.
  tf.compat.v1.nn.rnn_cell.GRUCell(self.hidden_size),
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/content/AIPI-531-Final-Project/H_and_M_Data/src/SNQN_with_features.py:210: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  self.output